In [1]:
from collections import Counter
from sklearn.datasets import load_breast_cancer
from imblearn.under_sampling import RandomUnderSampler

X,y = load_breast_cancer(True)

#Fix class imbalance with under sampling
rus = RandomUnderSampler()
Xb,yb = rus.fit_resample(X, y)

print('Original dataset shape %s' % Counter(y))
print('Resampled dataset shape %s' % Counter(yb))

Original dataset shape Counter({1: 357, 0: 212})
Resampled dataset shape Counter({0: 212, 1: 212})


In [2]:
X,y = Xb,yb

#scale data into [0,1] range
from sklearn import preprocessing
mms = preprocessing.MinMaxScaler()
Xs = mms.fit_transform(X)

In [3]:
X = Xs

#split data into train / test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


print("Test : ",X_test.shape,y_test.shape)
print("Train : ",X_train.shape,y_train.shape)

Test :  (85, 30) (85,)
Train :  (339, 30) (339,)


In [7]:
import numpy as np

class dlnet:
    def __init__(self, x, y):
        self.X=x
        self.Y=y
        self.Yh=np.zeros((1,self.Y.shape[1]))
        
        self.n_layer=2 #number of layers
        self.dims = [9, 15, 1] #size of each layer 
        self.param = {} #used to store weights
        self.temp = {}  #used to store temp values
        self.grad = {}
        self.loss = []
        self.lr=0.003
        self.sam = self.Y.shape[1]
        
    def nInit(self):    
        np.random.seed(1)
        self.param['W1'] = np.random.randn(self.dims[1], self.dims[0]) / np.sqrt(self.dims[0]) 
        self.param['b1'] = np.zeros((self.dims[1], 1))        
        self.param['W2'] = np.random.randn(self.dims[2], self.dims[1]) / np.sqrt(self.dims[1]) 
        self.param['b2'] = np.zeros((self.dims[2], 1))                
        return
    
    #Activation Functions
    def Sigmoid(Z):
        return 1/(1+np.exp(-Z))
    
    def Relu(Z):
        return np.maximum(0,Z)
    
    def Tanh(Z):
        return np.tanh(Z)
    
    #Feed Forward
    def forward(self):    
        Z1 = self.param['W1'].dot(self.X) + self.param['b1'] 
        A1 = Relu(Z1)
        self.temp['Z1']=Z1
        self.temp['A1']=A1
        
        Z2 = self.param['W2'].dot(A1) + self.param['b2']  
        A2 = Sigmoid(Z2)
        self.ch['Z2']=Z2
        self.ch['A2']=A2
        self.Yh=A2
        loss=self.nloss(A2)
        return self.Yh, loss
    
    #Cross Entropy Loss Function
    def nloss(self,Yh):
        loss = (1./self.sam) * (-np.dot(self.Y,np.log(Yh).T) - np.dot(1-self.Y, np.log(1-Yh).T))    
        return loss
    
    #Sum of Squared Error Loss Function
    def sseloss(self,Yh):
        squared_errors = (self.Yh - self.Y) ** 2
        loss = np.sum(squared_errors)
        return loss
    
    #Derivative of Activation Functions
    def dRelu(x):
        x[x<=0] = 0
        x[x>0] = 1
        return x
    
    def dSigmoid(Z):
        s = 1/(1+np.exp(-Z))
        dZ = s * (1-s)
        return dZ
    
    #Back propagation
    def backward(self):
        dLoss_Yh = - (np.divide(self.Y, self.Yh ) - np.divide(1 - self.Y, 1 - self.Yh))    
        
        dLoss_Z2 = dLoss_Yh * dSigmoid(self.temp['Z2'])    
        dLoss_A1 = np.dot(self.param["W2"].T,dLoss_Z2)
        dLoss_W2 = 1./self.temp['A1'].shape[1] * np.dot(dLoss_Z2,self.temp['A1'].T)
        dLoss_b2 = 1./self.temp['A1'].shape[1] * np.dot(dLoss_Z2, np.ones([dLoss_Z2.shape[1],1])) 
                            
        dLoss_Z1 = dLoss_A1 * dRelu(self.ch['Z1'])        
        dLoss_A0 = np.dot(self.param["W1"].T,dLoss_Z1)
        dLoss_W1 = 1./self.X.shape[1] * np.dot(dLoss_Z1,self.X.T)
        dLoss_b1 = 1./self.X.shape[1] * np.dot(dLoss_Z1, np.ones([dLoss_Z1.shape[1],1]))  
        
        self.param["W1"] = self.param["W1"] - self.lr * dLoss_W1
        self.param["b1"] = self.param["b1"] - self.lr * dLoss_b1
        self.param["W2"] = self.param["W2"] - self.lr * dLoss_W2
        self.param["b2"] = self.param["b2"] - self.lr * dLoss_b2
    
    #Gradient descent
    def gd(self,X, Y, iter = 3000):
        np.random.seed(1)                         
    
        self.nInit()
    
        for i in range(0, iter):
            Yh, loss=self.forward()
            self.backward()
        
            if i % 500 == 0:
                print ("Cost after iteration %i: %f" %(i, loss))
                self.loss.append(loss)
    
        return

In [11]:
nn = dlnet(X_train,y_train)
nn.gd(X_train, y_train, iter = 15000)
#X_train.shape

IndexError: tuple index out of range